In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mangoes/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/mangoes/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/mangoes/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 136 images belonging to 2 classes.
Found 33 images belonging to 2 classes.
Found 14 images belonging to 1 classes.


In [3]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
5/5 [==============================] - 47s 8s/step - loss: 2.4261 - accuracy: 0.5882 - val_loss: 0.7430 - val_accuracy: 0.5455
Epoch 2/5
5/5 [==============================] - 16s 3s/step - loss: 0.4208 - accuracy: 0.8382 - val_loss: 0.6376 - val_accuracy: 0.4848
Epoch 3/5
5/5 [==============================] - 16s 3s/step - loss: 0.1737 - accuracy: 0.9853 - val_loss: 1.1051 - val_accuracy: 0.5758
Epoch 4/5
5/5 [==============================] - 15s 3s/step - loss: 0.0776 - accuracy: 0.9706 - val_loss: 1.6958 - val_accuracy: 0.6061
Epoch 5/5
5/5 [==============================] - 16s 3s/step - loss: 0.0711 - accuracy: 0.9632 - val_loss: 0.6241 - val_accuracy: 0.8182


In [6]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
model = load_model('/content/Model.h5')
class_names = ['ripen mango','raw mango']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [16]:
image = Image.open("/content/drive/MyDrive/mangoes/train/Ripe-20240215T012914Z-001/Ripe/1 (37).jpg").convert("RGB")
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
data[0] = normalized_image_array

In [20]:
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)

1/1 [==============================] - 0s 76ms/step
Result:  ripen mango

Accuracy:  4.5687606e-10
